In [1]:
import numpy as np
from __future__ import print_function
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [23]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [59]:
batch_size = 16
kernel_size = 5
num_hidden_units = 1024
depth = 64


graph = tf.Graph()
with graph.as_default():
    training_data = tf.placeholder(np.float32, [batch_size,image_size,image_size,num_channels])
    training_labels = tf.placeholder(np.float32, [batch_size,num_labels])
    validation_data = tf.constant(valid_dataset)
    validation_labels = tf.constant(valid_labels)
    testing_data = tf.constant(test_dataset)
    testing_labels = tf.constant(test_labels)
    
    weights_10 = tf.Variable(tf.truncated_normal([kernel_size, kernel_size,num_channels,depth//2],stddev=0.1))
    biases_0 = tf.Variable(tf.constant(1.0, shape=[depth//2]))
    
    weights_21 = tf.Variable(tf.truncated_normal([kernel_size, kernel_size,depth//2,depth],stddev=0.1))
    biases_1 = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    weights_32 = tf.Variable(tf.truncated_normal([image_size//4 * image_size//4 * depth,num_hidden_units],stddev=0.1))
    biases_2 = tf.Variable(tf.constant(1.0, shape=[num_hidden_units]))
    
    weights_43 = tf.Variable(tf.truncated_normal([num_hidden_units,num_labels]))
    biases_3 = tf.Variable(tf.constant(1.0, shape =[num_labels]))
    
    def model(data, isTrain):
        conv2d_1 = tf.nn.relu(tf.nn.conv2d(data,weights_10,[1,1,1,1],"SAME")+biases_0)
        conv2d_1 = tf.nn.max_pool(conv2d_1, [1,2,2,1], [1,2,2,1], "SAME")
        conv2d_2 = tf.nn.relu(tf.nn.conv2d(conv2d_1,weights_21,[1,1,1,1],"SAME")+biases_1)
        conv2d_2 = tf.nn.max_pool(conv2d_2, [1,2,2,1], [1,2,2,1], "SAME")
        shape = conv2d_2.get_shape().as_list()
        reshape = tf.reshape(conv2d_2, [shape[0],shape[1]*shape[2]*shape[3]])
        hidden_layer = tf.nn.relu(tf.matmul(reshape,weights_32)+biases_2)
        if isTrain:
            hidden_layer = tf.nn.dropout(hidden_layer,keep_prob=0.5)
        output_layer = tf.matmul(hidden_layer,weights_43)+biases_3
        return output_layer
    
    logits = model(training_data,False)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=training_labels))
    
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    training_predictions = tf.nn.softmax(logits)
    validation_predictions = tf.nn.softmax(model(validation_data,False))
    testing_predictions = tf.nn.softmax(model(testing_data,False))
    
        

In [ ]:
num_steps = 1001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_training_data = train_dataset[offset:(offset+batch_size)]
        batch_training_labels = train_labels[offset:(offset+batch_size)]
        feed_dict = {training_data : batch_training_data, training_labels : batch_training_labels}
        _,l,predictions = session.run([optimizer,loss,training_predictions],feed_dict=feed_dict)
        if step % 100 ==0:
            print("Loss at training step %d: %f" % (step,l))
            print("Training accuracy: %.1f%%" % accuracy(predictions, batch_training_labels))
            print("Validation accuracy: %.1f%%\n" % accuracy(validation_predictions.eval(),valid_labels))
    print("Testing accuracy: %.1f%%" % accuracy(testing_predictions.eval(),test_labels))

Initialized
Loss at training step 0: 327.814850
Training accuracy: 12.5%
Validation accuracy: 10.0%

Loss at training step 100: nan
Training accuracy: 0.0%
Validation accuracy: 10.0%

